In [1]:
import codecs
from bs4 import BeautifulSoup
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import random, sys
import pandas as pd
from urllib.parse import quote_plus

Using TensorFlow backend.


In [2]:
# 파일을 읽어온다. 
fp = pd.read_csv('all_event.csv')

import csv
with open('all_event.csv' , 'rt') as fin:
    cin = csv.reader(fin) 
    all_event = [row for row in cin]
    
print(fp.shape)
# 사건갯수, 시간

(50000, 24)


In [3]:
results_ = []
for i , event in enumerate(all_event):
    # print(i , event)
    event = event[1:]
    
    if '' in event:
        event = event[:event.index('')]
        
    evt_txt = ' '.join(event) 
    results_.append(evt_txt)

In [4]:
results = results_[1:]
results

['No3368 En1339 Po1313 Te1354 En2411 No2563 Po3339 Bu2471 Ec3212 Se2421',
 'En1314 En1442 Se3242 Tr3139 Tr2346 Se3233 En1434 No1366 No2127 No1433 Tr1241 Se3252 Se3514',
 'No2112 Tr3443 Se1248 No2271 Se3329 Tr3257 Te2114 Ec3234 Tr1476 Te3367 Po2178 Se1335 Se3141 Tr1321 Po2559 Po2533 Te1441',
 'Bu1271 En3567 Te3328 Bu2155 Po2468 Se1428 Bu3143 Ec1262 Bu1435 Bu3511 No3478',
 'Se1126 Tr3317 Po2149 Te3473 En2146 En2335 No2457 Po3128 Ec2221 Tr1116',
 'Se1534 Po1237 Tr3568 No2458 Se2471 Se2525 Bu1554 Ec2325 En3462 Ec2258 Bu3242 Bu3359 Tr1411 En2526 Te3274 Te3357 No2353 No1143 Po1513 Te1377 Ec1439 No1255 Ec3422',
 'En2241 Te3472 No2358 Ec2518 En1317',
 'Se1311 En3523 Se2539 No3454 Tr3539 Se2214 Bu1454 No1325 Po3137 Tr2271 Bu3477',
 'No3459 Te3265 Bu1121 Bu1466 Tr3129 Te2252 Tr3531',
 'En3126 Bu1348 Te3453 Se1544 Se2469 Bu2551 Tr2439 Te3252',
 'Te1544 Se1413 No3162 Po3451 En1458 Po3447 Tr2547 Tr2457 Tr2575 Po1566 Se1236 Po1159 Te2132 Te2128',
 'Te3537 Te1564 Se1523 No3239 Ec2161 Te1177 Po2313 No

In [5]:
all_text = " ".join(results) 
text = all_text
print('훈련 데이터: ', len(text))

# 사건사전 생성
text_split = text.split(" ")
chars = sorted(list(set(text_split)))
print('사건의 종류:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))  
indices_char = dict((i, c) for i, c in enumerate(chars))  

# 전체 훈련 데이터를 쪼개어서 문장구조처럼 만들기
maxlen = 20  # 예측할 사건 시간 범위
step = 10
sentences = []
next_chars = []

# sentences에 maxlen개씩 엮어넣고
# next_chars에 그 다음 사건 넣기
for i in range(0, len(text_split) - maxlen, step):
    sentences.append(" ".join(text_split[i: i + maxlen]))
    next_chars.append(text_split[i + maxlen])
print("sentences:", len(sentences))


훈련 데이터:  4906670
사건의 종류: 7560
sentences: 70094


In [6]:
print('학습할 사건들의 수:', len(sentences))
X = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(next_chars), len(chars)), dtype=np.bool)
print("X:", X.shape)
print("y:", y.shape)

for i, sentence in enumerate(sentences):  # 학습할 사건들
    # one-hot encoding
    for t, char in enumerate(sentence.split(" ")):
        X[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

학습할 사건들의 수: 70094
X: (70094, 20, 7560)
y: (70094, 7560)


In [8]:
# 모델 구축하기(LSTM)
print('LSTM 모델을 만들어 본다....')
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))  # 일련의 사건 목록 수, 사건 종류 수
model.add(Dense(len(chars)))
model.add(Activation('softmax'))
optimizer = RMSprop(lr=0.015)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)


model.fit(X, y, batch_size=128, epochs=5)

# for문 대신 epochs를 늘림

LSTM 모델을 만들어 본다....
Epoch 1/5
70094/70094 [==============================] - 201s 3ms/step - loss: 9.0944
Epoch 2/5
70094/70094 [==============================] - 199s 3ms/step - loss: 8.9841
Epoch 3/5
70094/70094 [==============================] - 203s 3ms/step - loss: 8.8573
Epoch 4/5
70094/70094 [==============================] - 201s 3ms/step - loss: 8.9128
Epoch 5/5
70094/70094 [==============================] - 199s 3ms/step - loss: 8.9749


In [ ]:
# 예측사건의 검증을 위한 적당한 threshold를 찾는다. 
next_probs = []

# 사건 일부 추출
all_s = [] 
for i, s in enumerate(sentences):
    #print(i,s)
    if i%15 == 0:
        all_s.append(s)


for ix, sentence in enumerate(all_s):
    print(ix, sentence)
    
    # 20개만 생성
    if ix > 20:
        break
    
    # 일련의 사건 -> 리스트로
    x = np.zeros((1, maxlen, len(chars)))
    list_sen = sentence.split(" ")
    enum_sen = enumerate(list_sen)

    for t, char in enum_sen:
        x[0, t, char_indices[char]] = 1.  # 한 일련의 사건
        # 다음 사건 예측
        preds = model.predict(x, verbose=0)[0]  # 다음 사건 인덱스
        next_pred_char = indices_char[np.argmax(preds)]  # 다음 사건 코드

        # 뒤에 사건이 비어있을 때만 예측
        if t < len(list_sen)-1:
            next_char = list_sen[t+1]
            next_prob = preds[char_indices[next_char]]
            next_probs.append(next_prob)
        else:
            next_char = "END"
            next_prob = 0

        print(t, char, preds, next_char,
              next_pred_char, next_prob,
              (next_prob - np.average(preds))>0)

In [ ]:
# 일어난 사건을 검증하여 본다. 
sentence = "Tr3317 Po2149 Te3473 En2146"
x = np.zeros((1, maxlen, len(chars)))
list_sen = sentence.split(" ")
enum_sen = enumerate(list_sen)
err_word_all = []
real_word_all = []

In [ ]:
# 일반적으로 일어나는 사건의 확률의 정도를 감안하기 위하여 평균을 구하여 본다. 
for t, char in enum_sen:
    x[0, t, char_indices[char]] = 1.
    preds = model.predict(x, verbose=0)[0]
    next_pred_char = indices_char[np.argmax(preds)]

    if t < len(list_sen)-1:
        next_char = list_sen[t+1]
        next_prob = preds[char_indices[next_char]]
    else:
        next_char = "END"
        next_prob = 0

    diff = (next_prob - np.average(next_probs))
    
    if diff < 0:
        err_word = char + " " + next_char 
        err_word_all.append(err_word)
    elif diff > 0:
        real_word = char + " " + next_char
        real_word_all.append(real_word)
    
    print(t , char , next_char , next_prob , diff , diff > 0)

print(real_word_all, err_word_all[:-1])
